In [1]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv("churn-5000.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)


#BOX-COX
lemda=0.5
num_df=(num_df**lemda)
num_df=num_df-1
num_df[num_df < 0]=0
num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
#num_df=round(np.log(num_df.add(1)),2)
#num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log

num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [3]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X_train,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

from sklearn.model_selection import cross_val_score
roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
print('roc_auc: ',roc_auc.mean())

from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_train, y_pred)
auc = auc(fpr, tpr)
print('auc: ',auc)

3979 314 446 261
pod:  0.36916548797736914
pof:  0.07314232471465176
AUC:  0.6480115816313587
roc_auc:  0.8124706530504107
auc:  0.6480115816313587


In [4]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4203 90 572 135
pod:  0.19094766619519093
pof:  0.020964360587002098
AUC:  0.5849916528040945


In [5]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4291 2 697 10
pod:  0.014144271570014143
pof:  0.0004658746797111577
AUC:  0.5068391984451515


In [6]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression()
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)   

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

4215 78 598 109
pod:  0.15417256011315417
pof:  0.01816911250873515
AUC:  0.5680017238022095


In [3]:
#SVM
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

4293 0 707 0
pod:  0.0
pof:  0.0
AUC:  0.5


In [7]:
#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(X_train,y_train):
    X2_train, X2_test, y2_train, y2_test=X_train.iloc[train_index], X_train.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print ('tn mean: ',sum(tn_list) / len(tn_list))
print ('fp mean: ',sum(fp_list) / len(fp_list))
print ('fn mean: ',sum(fn_list) / len(fn_list))
print ('tp mean: ',sum(tp_list) / len(tp_list))

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 223us/step - loss: 2.2618 - acc: 0.8561 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 115us/step - loss: 2.2796 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 3/30
3599/3599 [==============================] - 0s 115us/step - loss: 2.2796 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 4/30
3599/3599 [==============================] - 0s 115us/step - loss: 2.2796 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
y2_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 286us/step - loss: 0.4885 - acc: 0.8472 - val_loss: 0.4696 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 107us/step - loss: 0.4344 - acc: 0.8583 - val_loss: 0.4303 - val_acc: 0.8589
Epoch 3/30
3599/3599 [==============================] - 0s 103us/step - loss: 0.4239 - acc: 0.8586 - val_loss: 0.4261 - val_acc: 0.8589
Epoch 4/30
3599/3599 [==============================] - 0s 112us/step - loss: 0.4181 - acc: 0.8586 - val_loss: 0.4127 - val_acc: 0.8589
Epoch 5/30
3599/3599 [==============================] - 0s 119us/step - loss: 0.4149 - acc: 0.8583 - val_loss: 0.4085 - val_acc: 0.8589
Epoch 6/30
3599/3599 [==============================] - 0s 115us/step - loss: 0.4119 - acc: 0.8586 - val_loss: 0.4116 - val_acc: 0.8567
Epoch 7/30
3599/3599 [==============================] - 0s 121us/step - loss: 0.4051 - acc: 0.8580 - val_loss: 0.4054 - val_acc: 0.8589
E

Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 311us/step - loss: 0.5394 - acc: 0.8469 - val_loss: 0.4424 - val_acc: 0.8589
Epoch 2/30
3600/3600 [==============================] - 0s 111us/step - loss: 0.4370 - acc: 0.8586 - val_loss: 0.4220 - val_acc: 0.8589
Epoch 3/30
3600/3600 [==============================] - 0s 106us/step - loss: 0.4327 - acc: 0.8586 - val_loss: 0.4299 - val_acc: 0.8589
Epoch 4/30
3600/3600 [==============================] - 0s 106us/step - loss: 0.4279 - acc: 0.8589 - val_loss: 0.4205 - val_acc: 0.8589
Epoch 5/30
3600/3600 [==============================] - 0s 108us/step - loss: 0.4200 - acc: 0.8586 - val_loss: 0.4169 - val_acc: 0.8589
Epoch 6/30
3600/3600 [==============================] - 0s 118us/step - loss: 0.4180 - acc: 0.8586 - val_loss: 0.4145 - val_acc: 0.8589
Epoch 7/30
3600/3600 [==============================] - 0s 107us/step - loss: 0.4172 - acc: 0.8586 - val_loss: 0.4167 - val_acc: 0.8589
E

 [0.]]
pod 1st:  0.0
Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 365us/step - loss: 2.4565 - acc: 0.8422 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 2/30
3600/3600 [==============================] - 0s 131us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 3/30
3600/3600 [==============================] - 0s 121us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 4/30
3600/3600 [==============================] - 0s 131us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
y2_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]


Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 412us/step - loss: 2.3184 - acc: 0.8497 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 2/30
3600/3600 [==============================] - 0s 126us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 3/30
3600/3600 [==============================] - 0s 130us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 4/30
3600/3600 [==============================] - 0s 125us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
y2_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 2s 437us/step - loss: 2.3100 - acc: 0.8508 - val_loss: 2.1825 - val_acc: 0.8646
Epoch 2/30
3600/3600 [==============================] - 0s 102us/step - loss: 2.3058 - acc: 0.8569 - val_loss: 2.1825 - val_acc: 0.8646
Epoch 3/30
3600/3600 [==============================] - 0s 104us/step - loss: 2.3058 - acc: 0.8569 - val_loss: 2.1825 - val_acc: 0.8646
Epoch 4/30
3600/3600 [==============================] - 0s 104us/step - loss: 2.3058 - acc: 0.8569 - val_loss: 2.1825 - val_acc: 0.8646
y2_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

In [8]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(X_train,y_train):
    X2_train, X2_test, y2_train, y2_test=X_train.iloc[train_index], X_train.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print ('tn mean: ',sum(tn_list) / len(tn_list))
print ('fp mean: ',sum(fp_list) / len(fp_list))
print ('fn mean: ',sum(fn_list) / len(fn_list))
print ('tp mean: ',sum(tp_list) / len(tp_list))

Train on 3599 samples, validate on 900 samples
Epoch 1/5
3599/3599 [==============================] - 2s 568us/step - loss: 0.4894 - acc: 0.8550 - val_loss: 0.4034 - val_acc: 0.8589
Epoch 2/5
3599/3599 [==============================] - 1s 173us/step - loss: 0.4028 - acc: 0.8586 - val_loss: 0.3989 - val_acc: 0.8589
Epoch 3/5
3599/3599 [==============================] - 1s 172us/step - loss: 0.3901 - acc: 0.8586 - val_loss: 0.3885 - val_acc: 0.8589
Epoch 4/5
3599/3599 [==============================] - 1s 181us/step - loss: 0.3785 - acc: 0.8586 - val_loss: 0.3912 - val_acc: 0.8589
Epoch 5/5
3599/3599 [==============================] - 1s 189us/step - loss: 0.3684 - acc: 0.8586 - val_loss: 0.4075 - val_acc: 0.8589
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 3600 samples, validate on 900 samples
Epoch 1/5
3600/3600 [==============================] - 4s 988us/step - loss: 0.4692 - acc: 0.8586 - val_loss: 0.4118 - val_acc: 0.8589
Epoch 2/5
3600/3600 [==============================] - 1s 232us/step - loss: 0.3980 - acc: 0.8586 - val_loss: 0.3968 - val_acc: 0.8589
Epoch 3/5
3600/3600 [==============================] - 1s 247us/step - loss: 0.3874 - acc: 0.8586 - val_loss: 0.3898 - val_acc: 0.8589
Epoch 4/5
3600/3600 [==============================] - 1s 254us/step - loss: 0.3778 - acc: 0.8586 - val_loss: 0.3835 - val_acc: 0.8589
Epoch 5/5
3600/3600 [==============================] - 1s 237us/step - loss: 0.3689 - acc: 0.8586 - val_loss: 0.4000 - val_acc: 0.8589
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

In [9]:
#CNN - 1D CNN
def get_CNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras import layers
    from keras.optimizers import RMSprop
    max_features = 10000 # number of words to consider as features
    
    model = Sequential()
    #model.add(layers.Embedding(max_features, 128, input_length=96))
    model.add(layers.Embedding(max_features, 128, input_length=19))
    model.add(layers.Conv1D(32, 2, activation='relu'))
    model.add(layers.MaxPooling1D(5))
    model.add(layers.Conv1D(32, 2, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(1))
    model.summary()
    model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    y22_pred[y22_pred ==2.0]=1
   # y22_pred=y22_pred[y22_pred > 1.0]=1
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(X_train,y_train):
    X2_train, X2_test, y2_train, y2_test=X_train.iloc[train_index], X_train.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_CNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print ('tn mean: ',sum(tn_list) / len(tn_list))
print ('fp mean: ',sum(fp_list) / len(fp_list))
print ('fn mean: ',sum(fn_list) / len(fn_list))
print ('tp mean: ',sum(tp_list) / len(tp_list))



Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 19, 128)           1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 18, 32)            8224      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 32)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 32)             2080      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 1)                 33        
Total params: 1,290,337
Trainable params: 1,290,337
Non-trainable params: 0
__________________________________________

Train on 3599 samples, validate on 900 samples
Epoch 1/5
3599/3599 [==============================] - 3s 959us/step - loss: 0.4108 - acc: 0.8586 - val_loss: 0.4063 - val_acc: 0.8589
Epoch 2/5
3599/3599 [==============================] - 1s 296us/step - loss: 0.4010 - acc: 0.8586 - val_loss: 0.4039 - val_acc: 0.8589
Epoch 3/5
3599/3599 [==============================] - 1s 310us/step - loss: 0.3951 - acc: 0.8586 - val_loss: 0.4024 - val_acc: 0.8589
Epoch 4/5
3599/3599 [==============================] - 1s 291us/step - loss: 0.3901 - acc: 0.8586 - val_loss: 0.3999 - val_acc: 0.8589
Epoch 5/5
3599/3599 [==============================] - 1s 293us/step - loss: 0.3852 - acc: 0.8586 - val_loss: 0.3984 - val_acc: 0.8589
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Train on 3600 samples, validate on 900 samples
Epoch 1/5
3600/3600 [==============================] - 4s 1ms/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 2/5
3600/3600 [==============================] - 1s 314us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 3/5
3600/3600 [==============================] - 1s 318us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 4/5
3600/3600 [==============================] - 1s 331us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 5/5
3600/3600 [==============================] - 1s 330us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
y22_pred:  [[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-

Train on 3600 samples, validate on 900 samples
Epoch 1/5
3600/3600 [==============================] - 4s 1ms/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 2/5
3600/3600 [==============================] - 1s 274us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 3/5
3600/3600 [==============================] - 1s 271us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 4/5
3600/3600 [==============================] - 1s 270us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
Epoch 5/5
3600/3600 [==============================] - 1s 279us/step - loss: 2.2789 - acc: 0.8586 - val_loss: 2.2744 - val_acc: 0.8589
y22_pred:  [[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-

Train on 3600 samples, validate on 901 samples
Epoch 1/5
3600/3600 [==============================] - 4s 1ms/step - loss: 0.7832 - acc: 0.8569 - val_loss: 0.4581 - val_acc: 0.8646
Epoch 2/5
3600/3600 [==============================] - 1s 324us/step - loss: 0.4550 - acc: 0.8569 - val_loss: 0.4205 - val_acc: 0.8646
Epoch 3/5
3600/3600 [==============================] - 1s 325us/step - loss: 0.4223 - acc: 0.8569 - val_loss: 0.4010 - val_acc: 0.8646
Epoch 4/5
3600/3600 [==============================] - 1s 326us/step - loss: 0.4079 - acc: 0.8569 - val_loss: 0.3975 - val_acc: 0.8646
Epoch 5/5
3600/3600 [==============================] - 1s 328us/step - loss: 0.4026 - acc: 0.8569 - val_loss: 0.3959 - val_acc: 0.8646
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]